$u_\beta^*= \argmin E(u)+\beta \mathcal L(u)$

$u_\beta^*-u_0^*=f(\beta)$?, $f(0.01) > \epsilon$?

In [ ]:
# instantiate model
from hydra import compose, initialize
from hydra_zen import instantiate, store

from configs import register_everything

store._overwrite_ok = True
overrrides = [
    "experiment=ep-xor",
    "model.positive_w=false",
    "model.bias=false",
    "model.normalize_weights=false",
    "model.clip_weights=false",
    "model.scale_output=2",
    "model.scale_input=1",
]
register_everything()
with initialize(config_path="../../configs", version_base="1.3"):
    cfg = compose(config_name="train", return_hydra_config=True, overrides=overrrides)

net2 = instantiate(cfg.model.net)()  #
dm = instantiate(cfg.data)
dm.setup()
dl = dm.train_dataloader()
print(net2)

In [ ]:
import torch

# load model checkpoint
ckpt = torch.load("../logs/debug/runs/2024-04-05_15-26-14/checkpoints/epoch=19-step=80.ckpt")
# get weights from lin1 & last layers
# w1 = ckpt["state_dict"]["net.model.0.weight"]
# w2 = ckpt["state_dict"]["net.model.1.weight"]
# get biases from lin1 & 1 layers
# b1 = ckpt["state_dict"]["net.model.0.bias"]
# b2 = ckpt["state_dict"]["net.model.1.bias"]
# get input & output dimensions

In [ ]:
net2.model = ckpt["state_dict"]["net.model"]

In [ ]:
# free phase
for x, y in dl:
    u_free = net2.solver(x)
    # TODO: loss backprop, decouple solver from model.ypred
    u_nudge = net2.solver(x, y)
    u_diff = torch.norm(u_nudge - u_free)
    print(u_diff)